In [460]:
M = Manifold(3, 'M', start_index=1)
X.<x,y,z> = M.chart()
# e = M.vector_frame('e')
# de = M.coframes()[1] 
eX = X.frame()
dX = X.coframe()



In [461]:
# Declaration of 0-form (scalar field)
f = M.scalar_field(x^2)
f.display()

# Evaluation of 0-form
p = M.point((1,2,3))
f(p)


1

In [480]:
# Declaration 1-form
a = M.diff_form(1, {eX:[x*y*z, 0, 0]}, name='a')
a.display(X)
# a[X, :] = [x*y*z, 0, x*y*z]
# a.display(eX)

# evaluation 1-form
# a(eX[1])(p)

a = x*y*z dx

In [570]:
# Declaration 2-form
b = M.diff_form(2, name='b')
b[eX, 1, 2] = x*y*z #dx^dy
b[eX, 1, 3] = y^2 #dx^dz
b[eX, 2, 3] = z*x^2 #dy^dz
b.display(X)


# evaluation 1-form
# p = M.point((1,2,3))
# b(eX[1], eX[2])(p)

b = x*y*z dx∧dy + y^2 dx∧dz + x^2*z dy∧dz

In [541]:
def numerical_sampler_kform(M, k_form, point):
    """
    Sample numerical values of a k-form at a point
    :param M: manifold
    :param k_form: k-form to sample
    :param point: point at which to sample
    :return: numerical values of the k-form at the point as a tensor
    """
    
    # Check if the k-form is a 0-form (scalar field)
    if k_form.degree() == 0: 
        p = M.point(point)
        return float(k_form(p))
    
    if k_form.degree() == 1:
        p = M.point(point)
        X.<x,y,z> = M.chart()
        eX = X.frame()
        return np.array([float(k_form(eX[i])(p)) for i in range(1,M.dimension()+1)])
    
    if k_form.degree() == 2:
        p = M.point(point)
        X.<x,y,z> = M.chart()
        eX = X.frame()
        return np.array([[float(k_form(eX[i], eX[j])(p)) for j in range(1,M.dimension()+1)] for i in range(1,M.dimension()+1)])

In [582]:
import numpy as np

def numerical_exterior_derivative(M, k_form, point, epsilon=1e-5):
    """
    Sample numerical values of the exterior derivative of a k-form at a point
    :param M: manifold
    :param k_form: k-form to sample
    :param point: point at which to sample
    :return: numerical values of the exterior derivative of the k-form at the point as a tensor
    """
    
    # If the k-form is a 0-form (scalar field)
    if k_form.degree() == 0: 
        dw = np.zeros(3)
        directions = [np.array(np.eye(3, dtype=int)[i]) for i in range(3)]
        point = np.array(point)
        for i in range(3):
            dw[i] = (1/(2*epsilon))*(numerical_sampler_kform(M, k_form, point + epsilon*directions[i]) - numerical_sampler_kform(M, k_form, point - epsilon*directions[i]))
        
        return dw
    
    # If the k-form is a 1-form
    elif k_form.degree() == 1:
        dw = np.zeros((3,3))
        directions = [np.array(np.eye(3, dtype=int)[i]) for i in range(3)]
        point = np.array(point)
        for i in range(3):
            for j in range(3):
                if i!=j:
                    dw[i,j] = (1/(2*epsilon))*(numerical_sampler_kform(M, k_form, point -  epsilon*directions[j])[i] +\
                                            numerical_sampler_kform(M, k_form, point +  epsilon*directions[i])[j] -\
                                            numerical_sampler_kform(M, k_form, point +  epsilon*directions[j])[i] -\
                                            numerical_sampler_kform(M, k_form, point -  epsilon*directions[i])[j])
        
        return dw
    
    elif k_form.degree() == 2:
        dw = np.zeros((3,3,3))
        directions = [np.array(np.eye(3, dtype=int)[i]) for i in range(3)]
        point = np.array(point)
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    if i!=j and j!=k and i!=k:
                        dw[i,j,k] = (1/(2*epsilon))*(+numerical_sampler_kform(M, k_form, point +  epsilon*directions[i])[j,k] -\
                                            numerical_sampler_kform(M, k_form, point -  epsilon*directions[i])[j,k] -\
                                            numerical_sampler_kform(M, k_form, point +  epsilon*directions[j])[i,k] +\
                                            numerical_sampler_kform(M, k_form, point -  epsilon*directions[j])[i,k] +\
                                            numerical_sampler_kform(M, k_form, point +  epsilon*directions[k])[i,j] -\
                                            numerical_sampler_kform(M, k_form, point -  epsilon*directions[k])[i,j])
                                            
        return dw
        
        

In [583]:
b.exterior_derivative().comp(X.frame())[:, X]

[[[0, 0, 0], [0, 0, (x - 2)*y + 2*x*z], [0, -(x - 2)*y - 2*x*z, 0]],
 [[0, 0, -(x - 2)*y - 2*x*z], [0, 0, 0], [(x - 2)*y + 2*x*z, 0, 0]],
 [[0, (x - 2)*y + 2*x*z, 0], [-(x - 2)*y - 2*x*z, 0, 0], [0, 0, 0]]]

In [573]:
numerical_exterior_derivative(M, b, (1,2,3))

array([[[ 0.,  0.,  0.],
        [ 0.,  0.,  4.],
        [ 0., -4.,  0.]],

       [[ 0.,  0., -4.],
        [ 0.,  0.,  0.],
        [ 4.,  0.,  0.]],

       [[ 0.,  4.,  0.],
        [-4.,  0.,  0.],
        [ 0.,  0.,  0.]]])

In [574]:
k_form = b
np.array(k_form.exterior_derivative().comp(X.frame())[:, X]).flatten()   

array([0, 0, 0, 0, 0, (x - 2)*y + 2*x*z, 0, -(x - 2)*y - 2*x*z, 0, 0, 0,
       -(x - 2)*y - 2*x*z, 0, 0, 0, (x - 2)*y + 2*x*z, 0, 0, 0,
       (x - 2)*y + 2*x*z, 0, -(x - 2)*y - 2*x*z, 0, 0, 0, 0, 0],
      dtype=object)

In [554]:
def test_numerical_approximation(M, k_form, point, epsilon=1e-3):
    """
    Test the numerical approximation of the exterior derivative of a k-form at a point
    :param M: manifold
    :param k_form: k-form to sample
    :param point: point at which to sample
    :return: True if the numerical approximation is close to the analytical value, False otherwise
    """
    def eval_symbolic_array(exprs, point):
        values = {x: point[0], y: point[1], z: point[2]}
        return np.array([float(SR(str(e)).substitute(values)) for e in exprs])

    
    # Compute the numerical exterior derivative
    numerical_derivative = np.array(numerical_exterior_derivative(M, k_form, point, epsilon=epsilon)).flatten()
    
    X.<x,y,z>= M.chart()
    analytic_coomp = np.array(k_form.exterior_derivative().comp(X.frame())[:, X]).flatten()     
    analytical_derivative = eval_symbolic_array(analytic_coomp, point)
    
    print("Flatten numerical derivative: ", numerical_derivative)
    print("Flatten analytical derivative: ", analytical_derivative)
    
    return np.allclose(numerical_derivative, analytical_derivative, atol=epsilon)     
        

In [581]:
test_numerical_approximation(M, b, (-32,221,986))

Flatten numerical derivative:  [     0.              0.              0.              0.
      0.         -70617.99999978      0.          70617.99999951
      0.              0.              0.          70617.99999978
      0.              0.              0.         -70617.99999955
      0.              0.              0.         -70617.99999975
      0.          70617.99999973      0.              0.
      0.              0.              0.        ]
Flatten analytical derivative:  [     0.      0.      0.      0.      0. -70618.      0.  70618.      0.
      0.      0.  70618.      0.      0.      0. -70618.      0.      0.
      0. -70618.      0.  70618.      0.      0.      0.      0.      0.]


True